# Langchain Tools and Agents

In [1]:
%load_ext dotenv
%dotenv

In [2]:
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser

from langchain_openai.chat_models import ChatOpenAI

from langchain_community.utilities import WikipediaAPIWrapper
from langchain_community.tools import WikipediaQueryRun

In [3]:
wikipedia_api = WikipediaAPIWrapper()

In [4]:
wikipedia_api.run('Python')

'Page: Python (programming language)\nSummary: Python is a high-level, general-purpose programming language. Its design philosophy emphasizes code readability with the use of significant indentation.\nPython is dynamically type-checked and garbage-collected. It supports multiple programming paradigms, including structured (particularly procedural), object-oriented and functional programming. It is often described as a "batteries included" language due to its comprehensive standard library.\nGuido van Rossum began working on Python in the late 1980s as a successor to the ABC programming language, and he first released it in 1991 as Python 0.9.0. Python 2.0 was released in 2000. Python 3.0, released in 2008, was a major revision not completely backward-compatible with earlier versions. Python 2.7.18, released in 2020, was the last release of Python 2.\nPython consistently ranks as one of the most popular programming languages, and it has gained widespread use in the machine learning comm

In [5]:
wikipedia_tool = WikipediaQueryRun(api_wrapper=wikipedia_api)

In [6]:
wikipedia_tool.name

'wikipedia'

In [7]:
wikipedia_tool.description

'A wrapper around Wikipedia. Useful for when you need to answer general questions about people, places, companies, facts, historical events, or other subjects. Input should be a search query.'

In [8]:
wikipedia_tool.args

{'query': {'description': 'query to look up on wikipedia',
  'title': 'Query',
  'type': 'string'}}

In [9]:
response = wikipedia_tool.invoke({'query': 'Python'})

In [10]:
print(response)

Page: Python (programming language)
Summary: Python is a high-level, general-purpose programming language. Its design philosophy emphasizes code readability with the use of significant indentation.
Python is dynamically type-checked and garbage-collected. It supports multiple programming paradigms, including structured (particularly procedural), object-oriented and functional programming. It is often described as a "batteries included" language due to its comprehensive standard library.
Guido van Rossum began working on Python in the late 1980s as a successor to the ABC programming language, and he first released it in 1991 as Python 0.9.0. Python 2.0 was released in 2000. Python 3.0, released in 2008, was a major revision not completely backward-compatible with earlier versions. Python 2.7.18, released in 2020, was the last release of Python 2.
Python consistently ranks as one of the most popular programming languages, and it has gained widespread use in the machine learning community

In [11]:
TEMPLATE = '''
Turn the following user input into a Wikipedia search query. Don't answer the question:

{input}
'''

prompt_template = PromptTemplate.from_template(template = TEMPLATE)

In [12]:
chat = ChatOpenAI(model_name = 'gpt-4o-mini', 
                  temperature = 0,
                  seed=365)

In [13]:
chain = prompt_template | chat | StrOutputParser() | wikipedia_tool

In [14]:
chain.invoke({'input':'Who is the creator of the Python programming language?'})

'Page: Python (programming language)\nSummary: Python is a high-level, general-purpose programming language. Its design philosophy emphasizes code readability with the use of significant indentation.\nPython is dynamically type-checked and garbage-collected. It supports multiple programming paradigms, including structured (particularly procedural), object-oriented and functional programming. It is often described as a "batteries included" language due to its comprehensive standard library.\nGuido van Rossum began working on Python in the late 1980s as a successor to the ABC programming language, and he first released it in 1991 as Python 0.9.0. Python 2.0 was released in 2000. Python 3.0, released in 2008, was a major revision not completely backward-compatible with earlier versions. Python 2.7.18, released in 2020, was the last release of Python 2.\nPython consistently ranks as one of the most popular programming languages, and it has gained widespread use in the machine learning comm

## Creating a retriever and a Custom Tool

In [15]:
from langchain_community.vectorstores import Chroma
from langchain_openai.embeddings import OpenAIEmbeddings

from langchain_core.tools import tool
from langchain_core.tools import create_retriever_tool
from platform import python_version

In [16]:
vectorstore = Chroma(persist_directory="./test", embedding_function=OpenAIEmbeddings())

C:\Users\matsv\AppData\Local\Temp\ipykernel_14440\292454112.py:1: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  vectorstore = Chroma(persist_directory="./test", embedding_function=OpenAIEmbeddings())


In [17]:
retriever = vectorstore.as_retriever(search_type='mmr', search_kwargs={'k':3, 'lambda_mult': 0.7})
retriever_tool = create_retriever_tool(retriever, name="Introduction to Data and Data Science Course Lectures", description='''For any questions regarding the 
                                       Introduction to Data and Data Science course, you must use this tool.''')

In [18]:
retriever_tool

Tool(name='Introduction to Data and Data Science Course Lectures', description='For any questions regarding the \n                                       Introduction to Data and Data Science course, you must use this tool.', args_schema=<class 'langchain_core.tools.retriever.RetrieverInput'>, func=functools.partial(<function _get_relevant_documents at 0x0000018F5EF0B2E0>, retriever=VectorStoreRetriever(tags=['Chroma', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.chroma.Chroma object at 0x0000018F5EF03340>, search_type='mmr', search_kwargs={'k': 3, 'lambda_mult': 0.7}), document_prompt=PromptTemplate(input_variables=['page_content'], input_types={}, partial_variables={}, template='{page_content}'), document_separator='\n\n', response_format='content'), coroutine=functools.partial(<function _aget_relevant_documents at 0x0000018F5EF0B250>, retriever=VectorStoreRetriever(tags=['Chroma', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.chroma.Chroma o

In [19]:
retriever_tool.args

{'query': {'description': 'query to look up in retriever',
  'title': 'Query',
  'type': 'string'}}

In [20]:
print(retriever_tool.invoke("Could you list the programming languages a data scientist should know?"))

What about big data? Apart from R and Python, people working in this area are often proficient in other languages like Java or Scala. These two have not been developed specifically for doing statistical analyses, however they turn out to be very useful when combining data from multiple sources. All right! Let’s finish off with machine learning. When it comes to machine learning, we often deal with big data

Alright! So… How are the techniques used in data, business intelligence, or predictive analytics applied in real life? Certainly, with the help of computers. You can basically split the relevant tools into two categories—programming languages and software. Knowing a programming language enables you to devise programs that can execute specific operations. Moreover, you can reuse these programs whenever you need to execute the same action

Great! We hope we gave you a good idea about the level of applicability of the most frequently used programming and software tools in the field of 

In [46]:
@tool
def get_python_version() -> str:
    ''' Useful for questions regarding the version of Python currently used. '''
    return python_version()

In [22]:
get_python_version

StructuredTool(name='Custom tool', description='Useful for questions regarding the version of Python currently used.', args_schema=<class 'langchain_core.utils.pydantic.Custom tool'>, func=<function get_python_version at 0x0000018F505DBB50>)

In [23]:
get_python_version.invoke({})

'3.10.0'

## Langchain Hub

In [43]:
from langchain_community.utilities import WikipediaAPIWrapper
from langchain_community.tools import WikipediaQueryRun
from langchain_community.vectorstores import Chroma

from langchain_openai.embeddings import OpenAIEmbeddings
from langchain_openai.chat_models import ChatOpenAI

from langchain.tools.retriever import create_retriever_tool

from langchain import hub

from platform import python_version

from langchain.agents import (create_tool_calling_agent, 
                              AgentExecutor)

from langchain_core.prompts import ChatPromptTemplate

from langchain_core.prompts import SystemMessagePromptTemplate, MessagesPlaceholder, HumanMessagePromptTemplate

import typing
import langchain_core.messages

In [25]:
wikipedia_tool = WikipediaQueryRun(api_wrapper = WikipediaAPIWrapper())

In [26]:
vectorstore = Chroma(persist_directory = "./intro-to-ds-lectures", 
                     embedding_function = OpenAIEmbeddings())

retriever = vectorstore.as_retriever(search_type = 'mmr', 
                                     search_kwargs = {'k':3, 
                                                      'lambda_mult':0.7})

retriever_tool = create_retriever_tool(retriever = retriever, 
                                       name = "Introduction-to-Data-and-Data-Science-Course-Lectures", 
                                       description = '''For any questions regarding 
                                       the Introduction to Data and Data Science course, you must use this tool.''')

In [27]:
chat = ChatOpenAI(model_name = 'gpt-4o-mini', 
                  temperature = 0,
                  seed=365)

In [28]:
chat_prompt_template = hub.pull("hwchase17/openai-tools-agent")

In [45]:
chat_prompt_template

ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], optional_variables=['chat_history'], input_types={'chat_history': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag(tag='HumanMessageChunk')], typing.Annotated[langchain_core.messages.chat.ChatMessageChunk, Tag(tag='ChatMessageChunk')], typing.Annotated[langchain_core.messages.system.SystemMessageChunk, Tag(tag='

In [30]:
chat_prompt_template.pretty_print()

================================ System Message ================================

You are a helpful assistant

============================= Messages Placeholder =============================

{chat_history}

================================ Human Message =================================

{input}

============================= Messages Placeholder =============================

{agent_scratchpad}


In [35]:
ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], 
                   input_types={'chat_history': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, 
                                                                         langchain_core.messages.human.HumanMessage, 
                                                                         langchain_core.messages.chat.ChatMessage, 
                                                                         langchain_core.messages.system.SystemMessage, 
                                                                         langchain_core.messages.function.FunctionMessage, 
                                                                         langchain_core.messages.tool.ToolMessage]], 
                                'agent_scratchpad': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, 
                                                                             langchain_core.messages.human.HumanMessage, 
                                                                             langchain_core.messages.chat.ChatMessage, 
                                                                             langchain_core.messages.system.SystemMessage, 
                                                                             langchain_core.messages.function.FunctionMessage, 
                                                                             langchain_core.messages.tool.ToolMessage]]}, 
                   messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], 
                                                                               template='You are a helpful assistant')), 
                             MessagesPlaceholder(variable_name='chat_history', optional=True), 
                             HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], 
                                                                              template='{input}')), 
                             MessagesPlaceholder(variable_name='agent_scratchpad')])

ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], optional_variables=['chat_history'], input_types={'chat_history': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]], 'agent_scratchpad': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]]}, partial_variables={'chat_history': []}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='You are a helpful assistant'), additional_kwargs={}), MessagesPlaceholder(variable_name='chat_history

In [47]:
tools = [wikipedia_tool, retriever_tool, get_python_version]

In [52]:
agent = create_tool_calling_agent(llm = chat, 
                                  tools = tools, 
                                  prompt = chat_prompt_template)

In [53]:
agent_executor = AgentExecutor(agent=agent, 
                               tools=tools, 
                               verbose=True, 
                               return_intermediate_steps=True)

In [54]:
response = agent_executor.invoke({"input":"Could you tell me the version of Python I'm currently using?"})



> Entering new AgentExecutor chain...

Invoking: `get_python_version` with `{}`


3.10.0You are currently using Python version 3.10.0.

> Finished chain.


StructuredTool(name='get_python_version', description='Useful for questions regarding the version of Python currently used.', args_schema=<class 'langchain_core.utils.pydantic.get_python_version'>, func=<function get_python_version at 0x0000018F6C140310>)